# Feature Extraction - Isomap 

알고리즘 출처 - 단단한 머신러닝 챕터 10 

**입력** 
- 샘플 세트 $D = {x_1, x_2, ..., x_n}$ 
- 근접 이웃 파라미터 k 
- 저차원 공간 차원수 d'

**과정**
1. for i = 1,2, ... n do 
- $x_i$의 근접 이웃 k를 설정 
- $x_i$와 K-최근접 이웃 사이의 거리를 유클리드 거리로 설정. 다른 점들과의 거리는 무한대로 설정 
- end for 

2. 최단거리 알고리즘을 사용하여 임의의 두 샘플 포인트 사이의 거리 ($x_1, x_j$)를 계산 
3. $dist(x_i, x_j)$를 MDS 알고리즘의 입력으로 설정 
4. return MDS 알고리즘의 출력

**출력**
- 샘플 세트 D의 저차원 공간에서의 투영 Z = ${z_1, z_2, ..., z_n}$

In [3]:
# 데이터 사용 및 라이브러리 설치 

import numpy as np
import pandas as pd
import random
import heapq

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression 
from collections import defaultdict

boston = load_boston()
X = boston.data 
y = boston.target
model = LinearRegression()

C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

### __init__ 및 1번 과정

**입력** 
- 샘플 세트 $D = {x_1, x_2, ..., x_m}$ 
- 근접 이웃 파라미터 k 
- 저차원 공간 차원수 d'

**과정**
1. for i = 1,2, ... m do 
- $x_i$의 근접 이웃 k를 설정 
- $x_i$와 K-최근접 이웃 사이의 거리를 유클리드 거리로 설정. 다른 점들과의 거리는 무한대로 설정 
- end for 

**구현해야하는 것** 
- 벡터간 유클리드 거리 계산 
- 유클리드 거리 상 짧은 순으로 K순위까지 선별 
- K 순위의 거리보다 낮은 경우 inf 값을 부여 


**필요한 것**
- X : 입력 데이터 
- k : 근접 이웃 개수 

**함수의 형태**
- def __init__(self, X,k,d) 

- def dist(self) : => $dist(x_i,x_j)$ metrix 

In [58]:
## __init__ , dist 함수 구현 

class Isomap() : 
    def __init__(self, X, k, d) : 
        self.X = X
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]
        
        self.k = k 
        self.d = d 
    
    def dist(self) : 
        dist_origin = [] 
        for i in range(self.n) : 
            origin = np.array(self.X[i]) 
            dist_vector = [np.linalg.norm(origin - np.array(self.X[j])) for j in range(self.n)]
            index = np.argsort(dist_vector)
            for idx, rank in enumerate(index) : 
                if rank >= self.k : dist_vector[idx] = float("inf") 
            dist_origin.append(dist_vector)
        return np.array(dist_origin)
            
        

In [63]:
test = Isomap(X, 4, 3)
a = test.dist()
print(a)

[[ 0.                 inf         inf ...         inf         inf
          inf]
 [58.9550797          inf 18.97631935 ...         inf         inf
          inf]
 [57.4920134          inf         inf ...         inf         inf
          inf]
 ...
 [        inf         inf         inf ...         inf         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]]


### 2. 최단거리 알고리즘을 사용하여 임의의 두 샘플 포인트 사이의 거리 ($x_1, x_j$)를 계산


**구현해야하는 것** 
- $x_i, x_j$ 간 최단거리 계산 : 다익스트라 알고리즘 적용 

**필요한 것**
- dist_metrix : n x n 거리 행렬 

**함수의 형태**
- def Dijkstra(self, dist_metrix) : => dist_metrix - 각 좌표간 거리 최단으로 변경 완료.  

In [66]:
from collections import defaultdict 
a = [[1,2,3], [4,3,2], [7,6,5]] 
graph = defaultdict(list)

for b,c,d in a : 
    graph[b].append((c,d))

print(graph)
    

defaultdict(<class 'list'>, {1: [(2, 3)], 4: [(3, 2)], 7: [(6, 5)]})


In [83]:
# 다익스트라 함수 추가 
class Isomap() : 
    def __init__(self, X, k, d) : 
        self.X = X
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]
        
        self.k = k 
        self.d = d 
    
    def dist(self) : 
        dist_origin = [] 
        for i in range(self.n) : 
            origin = np.array(self.X[i]) 
            dist_vector = [np.linalg.norm(origin - np.array(self.X[j])) for j in range(self.n)]
            index = np.argsort(dist_vector)
            for idx, rank in enumerate(index) : 
                if rank >= self.k : dist_vector[idx] = float("inf") 
            dist_origin.append(dist_vector)
        return np.array(dist_origin)
            
        
    def dijkstra(self) : 
        metrix = self.dist()
        graph = defaultdict(list) 
        for i in range(self.n) : 
            for j in range(self.n) : 
                graph[i].append((j, metrix[i,j]))
    
        for i in range(self.n) : 
            Q = [(0,i)]
            dist = defaultdict(int)
        
            while Q : 
                dis, node = heapq.heappop(Q)
                if node not in dist and dis != float("inf") : 
                    dist[node] = dis
                    for destination , add_dis in graph[node] : 
                        if add_dis != float("inf") : 
                            alt = dis + add_dis 
                            heapq.heappush(Q, (alt, destination))
                               
            for key, value in dist.items() : 
                metrix[i,j] = value 
    
        return metrix 

In [84]:
test = Isomap(X, 4, 3)
a = test.dist()
b = test.dijkstra()

print(b)

[[   0.                   inf           inf ...           inf
            inf 1315.1409526 ]
 [  58.9550797            inf   18.97631935 ...           inf
            inf 1259.52419294]
 [  57.4920134            inf           inf ...           inf
            inf 1350.45376302]
 ...
 [          inf           inf           inf ...           inf
            inf 1356.99272182]
 [          inf           inf           inf ...           inf
            inf 1284.63983006]
 [          inf           inf           inf ...           inf
            inf 1352.81593257]]


In [4]:
# 과거 MDS 파트 코딩한 내용 Isomap 클래스에 맞춰 수정 
# "svd did not converge" 오류 발생. 
class Isomap() : 
    def __init__(self, X, k, d) : 
        self.X = X
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]
        
        self.k = k 
        self.d = d 
    
    def dist(self) : 
        dist_origin = [] 
        for i in range(self.n) : 
            origin = np.array(self.X[i]) 
            dist_vector = [np.linalg.norm(origin - np.array(self.X[j])) for j in range(self.n)]
            index = np.argsort(dist_vector)
            for idx, rank in enumerate(index) : 
                if rank >= self.k : dist_vector[idx] = float("inf") 
            dist_origin.append(dist_vector)
        return np.array(dist_origin)
            
        
    def dijkstra(self, metrix) : 
        graph = defaultdict(list) 
        for i in range(self.n) : 
            for j in range(self.n) : 
                graph[i].append((j, metrix[i,j]))
    
        for i in range(self.n) : 
            Q = [(0,i)]
            dist = defaultdict(int)
        
            while Q : 
                dis, node = heapq.heappop(Q)
                if node not in dist and dis != float("inf") : 
                    dist[node] = dis
                    for destination , add_dis in graph[node] : 
                        if add_dis != float("inf") : 
                            alt = dis + add_dis 
                            heapq.heappush(Q, (alt, destination))
                               
            for key, value in dist.items() : 
                metrix[i,j] = value 
    
        return metrix 

    def dist_cal(self, dist_metrix) : 
        dist_i = dist_metrix.sum(axis=1)/self.n
        dist_j = dist_metrix.sum(axis=0)/self.n
        dist_all = dist_metrix.sum()/(self.n**2)
        
        return dist_metrix, dist_i, dist_j, dist_all
    
    def b(self, dist_metrix) : 
        metrix, row, column, sum_all = self.dist_cal(dist_metrix)
        return -(metrix - row.T - column + sum_all)/2 
    
    def eigen(self, dist_metrix) : 

        eigenvector_lst= []
        # dijkstra로 인해서 inf 값을 가진 원소 값이 많음. 
        # 임시로 inf의 값을 충분히 큰 1e+5 값으로 대체하여 계산하겠음. 
        np.where(np.array(dist_metrix) == float("inf"), 1e+5, dist_metrix)
        _, eigenvalue, eigenvector = np.linalg.svd(dist_metrix)
        index = np.argsort(eigenvalue)[::-1]
        for i, num in enumerate(index) : 
            eigenvector_lst.append(eigenvector[i, :])
        return np.take(eigenvalue, index), np.array(eigenvector_lst) 

    def mds(self, dist_metrix) : 
        eigenvalue, eigenvector = self.eigen(dist_metrix)
        eigenvalue_d_sqrt = np.sqrt(eigenvalue[:self.d])
        eigenvector_d = eigenvector[:self.d, :]
        return np.dot(np.diag(eigenvalue_d_sqrt), eigenvector_d)
    
    def isomap_goal(self) : 
        dist_metrix = self.dist() 
        dist_metrix = self.dijkstra(dist_metrix)
        
        return self.mds(dist_metrix)


In [5]:
test = Isomap(X, 4, 3)
b = test.isomap_goal()

print(b)

LinAlgError: SVD did not converge